In [32]:
def chunk_measurements(measurements):
    chunk_length = 5

    chunked_measurements = [measurements[i * chunk_length:(i + 1) * chunk_length] for i in range(
        (len(measurements) + chunk_length - 1) // chunk_length)]

    return chunked_measurements

In [42]:
def calc_avgs(measurements):
    avgs = {}
    for key in measurements[0].keys():
        avgs[key] = 0

    for measurement in measurements:
        for key in avgs.keys():
            avgs[key] += measurement[key]
            
    for key in avgs.keys():
        avgs[key] /= len(measurements)
        
    return avgs

In [63]:
def read_data(fp, start_index):
    temp_nums = []
    
    for line in fp:
        # skip empty lines (1 for \n)
        if len(line) == 0 or len(line) == 1:
            continue

        # skip failed attempts
        if "encrypt decrypt failed" in line or "begin" in line:
            continue

        # remove log info
        line = line[start_index:].strip()

        # remove end from beginning of line
        if "end" in line:
            line = line[5:]

        # extract number from line
        line = int(line[:line.index(" ")])

        # save number
        temp_nums.append(line)
            
    # group into measurements
    temp_nums = chunk_measurements(temp_nums)

    return temp_nums

In [51]:
def convert_enc(temp_nums):
    measurements = []
    # convert each to object
    for chunk in temp_nums:
        measurement = {
            "aes keygen": chunk[0],
            "aes encrypt": chunk[1],
            "elgamal encrypt": chunk[3],
            "total": chunk[4]
        }
        measurements.append(measurement)
        
    return measurements

In [54]:
def convert_dec(temp_nums):
    measurements = []
    # convert each to object
    for chunk in temp_nums:
        measurement = {
            "elgamal decrypt": chunk[1],
            "aes decrypt": chunk[3],
            "total": chunk[4]
        }
        measurements.append(measurement)
        
    return measurements

In [65]:
ENC_FILE = "encrypt_logs.txt"
DEC_FILE = "decrypt_logs.txt"
DIR_PATH = "./cryptography_timings/"

enc_measurements = []
dec_measurements = []

with open(DIR_PATH + ENC_FILE) as fp:
    enc_measurements = convert_enc(read_data(fp, 73))
    
with open(DIR_PATH + DEC_FILE) as fp:
    dec_measurements = convert_dec(read_data(fp, 80))


# size of data set
print("{num} Encryption Measurements".format(num=len(enc_measurements)))
print("{num} Decryption Measurements".format(num=len(dec_measurements)))

# find averages
display(calc_avgs(enc_measurements))
display(calc_avgs(dec_measurements))

727 Encryption Measurements
751 Decryption Measurements


{'aes keygen': 0.6189821182943603,
 'aes encrypt': 0.6258596973865199,
 'elgamal encrypt': 59.45804676753783,
 'total': 60.7111416781293}

{'elgamal decrypt': 61.81890812250333,
 'aes decrypt': 0.4234354194407457,
 'total': 62.254327563249}